In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import tensorflow as tf

In [ ]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        arr = pickle.load(f)
    return arr

In [ ]:
train = load_pickle('bouncing_balls_training_data.pkl')
train.shape

In [ ]:
plt.imshow(train[0,0,:,:],cmap=matplotlib.cm.Greys_r)

In [ ]:
sigma0_np = np.array([0.2, 0.4, 0.6], dtype=np.float32)*7
sigma0_np_mtx = np.diag(sigma0_np)
mu0_np = np.array([2, 4, 6], dtype=np.float32)

sigma1_np = np.array([0.1, 0.3, 0.5], dtype=np.float32)
sigma1_np_mtx = np.diag(sigma1_np)
mu1_np = np.array([1, 3, 5], dtype=np.float32)

k = 1.0

z1 = np.trace(np.matmul(np.linalg.inv(sigma1_np_mtx), sigma0_np_mtx))
z2 = np.matmul(np.matmul(np.transpose(mu1_np-mu0_np), np.linalg.inv(sigma1_np_mtx)), (mu1_np-mu0_np))
z3 = np.log(np.linalg.det(sigma1_np_mtx)/np.linalg.det(sigma0_np_mtx))
Dkl = 0.5 * (z1 + z2 - k + z3)
Dkl

In [ ]:
def kl_divergence_gaussians(q_mu, q_sigma, p_mu, p_sigma):
#https://hk.saowen.com/a/7404b78cd5b980e16e08423192e35ec18ecb3cb243d310a9d9a194747d9ee1ba    
    r = q_mu - p_mu
    return np.sum(np.log(p_sigma) - np.log(q_sigma) - .5 * (1. - (q_sigma**2 + r**2) / p_sigma**2), axis=-1)

In [ ]:
def kl_divergence_gaussians(q_mu, q_sigma, p_mu, p_sigma):
    r = q_mu - p_mu
    return np.sum(np.log(p_sigma) - np.log(q_sigma) - .5 * (1. - (q_sigma**2/p_sigma**2 + r**2/p_sigma**2)), axis=-1)

In [ ]:
def kl_divergence_gaussians_log(q_mu, q_sigma_log, p_mu, p_sigma_log):
    r = q_mu - p_mu
    return np.sum(p_sigma_log - q_sigma_log - .5 * (1. - (np.exp(q_sigma_log*2)/np.exp(p_sigma_log*2) + r**2/np.exp(p_sigma_log*2))), axis=-1)

In [ ]:
def kl_divergence_gaussians_tf(q_mu, q_sigma, p_mu, p_sigma):
    with tf.Session() as sess:
        zz = tf.reduce_sum(tf.distributions.kl_divergence(
        tf.distributions.Normal(loc=q_mu, scale=q_sigma),
        tf.distributions.Normal(loc=p_mu, scale=p_sigma)), axis=-1).eval()
    return zz

In [ ]:
def kl_divergence_gaussians_tf2(q_mu, q_sigma, p_mu, p_sigma):
#https://hk.saowen.com/a/7404b78cd5b980e16e08423192e35ec18ecb3cb243d310a9d9a194747d9ee1ba    
    with tf.Session() as sess:
        r = q_mu - p_mu
        return tf.reduce_sum(tf.log(p_sigma) - tf.log(q_sigma) - .5 * (1. - (q_sigma**2 + r**2) / p_sigma**2), axis=-1).eval()

In [ ]:
def kl_divergence_gaussians_tf3(q_mu, q_sigma, p_mu, p_sigma):
    with tf.Session() as sess:
        num = tf.square(q_mu - p_mu) + tf.square(q_sigma) - tf.square(p_sigma)
        den = 2 * tf.square(p_sigma) + 1e-8
        kl = tf.reduce_sum(num/den + tf.log(p_sigma) - tf.log(q_sigma), reduction_indices=[2,3,4])
        return kl.eval()


In [ ]:
#print(kl_divergence_gaussians(mu0_np, sigma0_np, mu1_np, sigma1_np))
print(kl_divergence_gaussians_tf(mu0_np, sigma0_np, mu1_np, sigma1_np))
print(kl_divergence_gaussians_tf3(mu0_np, sigma0_np, mu1_np, sigma1_np))
#print(kl_divergence_gaussians_tf2(mu0_np, sigma0_np, mu1_np, sigma1_np))
#print(kl_divergence_gaussians_log(mu0_np, np.log(sigma0_np), mu1_np, np.log(sigma1_np)))

In [ ]:
def gauss_KL(mu1, logstd1, mu2, logstd2):
    """ Returns KL divergence among two multivariate Gaussians, component-wise.

    It assumes the covariance matrix is diagonal. All inputs have shape (n,a).
    It is not necessary to know the number of actions because reduce_sum will
    sum over this to get the `d` constant offset. The part consisting of the
    trace in the formula is blended with the mean difference squared due to the
    common "denominator" of var2_na.  This forumula generalizes for an arbitrary
    number of actions.  I think mu2 and logstd2 should represent the policy
    before the update.

    Returns the KL divergence for each of the n components in the minibatch,
    then we do a reduce_mean outside this.
    """
    var1_na = tf.exp(2.*logstd1)
    var2_na = tf.exp(2.*logstd2)
    tmp_matrix = 2.*(logstd2 - logstd1) + (var1_na + tf.square(mu1-mu2))/var2_na - 1
    #kl_n = tf.reduce_sum(0.5 * tmp_matrix, axis=[1]) # Don't forget the 1/2 !!
    kl_n = 0.5 * tmp_matrix
    assert_op = tf.Assert(tf.reduce_all(kl_n >= -0.0000001), [kl_n]) 
    with tf.control_dependencies([assert_op]):
        kl_n = tf.identity(kl_n)
    return kl_n 

In [ ]:
def KL(mu_q, cov_q, mu_prior, cov_prior):
        """
        https://github.com/clinicalml/dmm/blob/master/model_th/dmm.py
        KL(q_t||p_t) = 0.5*(log|sigmasq_p| -log|sigmasq_q|  -D + Tr(sigmasq_p^-1 sigmasq_q)
                        + (mu_p-mu_q)^T sigmasq_p^-1 (mu_p-mu_q))
        """
        diff_mu = mu_prior-mu_q
        KL      = tf.log(cov_prior)-tf.log(cov_q) - 1. + cov_q/cov_prior + diff_mu**2/cov_prior
        KL_t    = 0.5*tf.reduce_sum(KL)
        return KL_t

In [ ]:
import tensorflow as tf

mu0 = tf.placeholder(tf.float32, shape=(3,))
sigma0 = tf.placeholder(tf.float32, shape=(3,))
mu1 = tf.placeholder(tf.float32, shape=(3,))
sigma1 = tf.placeholder(tf.float32, shape=(3,))


# https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
k = 1.0
tf1 = tf.reduce_sum(1.0/sigma1 * sigma0)
tf2 = tf.reduce_sum((mu1-mu0)*1.0/sigma1*(mu1-mu0))
tf3 = tf.cast(tf.log(tf.reduce_prod(sigma1/sigma0)), tf.float32)
Dkl = 0.5 * (tf1 + tf2 - k + tf3)

Dkl2 = tf.reduce_sum(gauss_KL(mu0, tf.diag(tf.log(sigma0)), mu1, tf.diag(tf.log(sigma1))))
Dkl3 = KL(mu0, sigma0, mu1, sigma1)

with tf.Session() as sess:
    
    sigma0_vec = np.array([0.2, 0.4, 0.6])
    mu0_vec = np.array([2, 4, 6])
    sigma1_vec = np.array([0.1, 0.3, 0.5])
    mu1_vec = np.array([1, 3, 5])

    dkl1, dkl2, dkl3 = sess.run([Dkl, Dkl2, Dkl3], feed_dict={mu0: mu0_vec, sigma0:sigma0_vec, mu1: mu1_vec, sigma1:sigma1_vec})
    print(dkl1, dkl2, dkl3)

In [ ]:
zz = tf.placeholder(tf.float32, shape=(15,))
yy = tf.split(zz, num_or_size_splits=3, axis=0)
with tf.Session() as sess:
    qq=sess.run(yy, feed_dict={zz:np.arange(0,15)})
print(qq)

In [ ]:
def initial_state_module(ex):
    """
    """
    with tf.variable_scope('initial_state_module'):
        shape = [-1,3]+ex.get_shape().as_list()[1:]
        print(shape)
        e = tf.reshape(ex, shape)
        ez = tf.unstack(e, axis=1)
        c = tf.concat(ez, axis=-1)
    return c

In [ ]:
ww = np.ones((10,10,1))
wwx = np.stack([ww*i for i in range(15)])
wwx.shape
ex = tf.placeholder(tf.float32, shape=(15,10,10,1))
qq = initial_state_module(ex)
with tf.Session() as sess:
    qq_=sess.run(qq, feed_dict={ex:wwx})
print(qq_.shape)
print(qq_[0,0,0,0],qq_[0,0,0,1],qq_[0,0,0,2])
print(qq_[1,0,0,0],qq_[1,0,0,1],qq_[1,0,0,2])
print(qq_[2,0,0,0],qq_[2,0,0,1],qq_[2,0,0,2])
print(qq_[3,0,0,0],qq_[3,0,0,1],qq_[3,0,0,2])
print(qq_[4,0,0,0],qq_[4,0,0,1],qq_[4,0,0,2])


In [ ]:
import train_env_model as mx
import tensorflow as tf
with tf.Session() as sess:
    with tf.variable_scope('env_model'):
        env_model = mx.EnvModel((13, 80, 80, 3), 1, 10)
        #env_model = EnvModel((13, 80, 80, 3), 1, 10)

    reg_loss = tf.reduce_sum(env_model.regularization_loss)
    rec_loss = tf.reduce_sum(env_model.reconstruction_loss)
    loss = reg_loss + rec_loss
    env_model.train_op = tf.train.AdamOptimizer().minimize(loss)

    sess.run(tf.global_variables_initializer())

    train = load_pickle('bouncing_balls_testing_data.pkl')
    train = np.expand_dims(train, 4)
    train = np.repeat(train, 3, axis=4)
    obs = train[0:2,:13,:,:,:]
    actions = np.zeros((2,13), dtype=np.float32)
    m = env_model

    print('Training')
    feed_dict = {env_model.obs:obs, env_model.actions:actions}
    #_, obs_hat_, next_state_, rec_loss_, reg_loss_, loss_ = sess.run([env_model.train_op, env_model.obs_hat, env_model.next_state, rec_loss, reg_loss, loss], feed_dict=feed_dict)
    obs_hat_, initial_state_, next_state_, reg_loss_, rec_loss_, eoi_, mu_, sigma_, mu_hat_, sigma_hat_, zz_ = sess.run(
      [m.obs_hat, m.initial_state, m.next_state, m.regularization_loss, m.reconstruction_loss, m.encoded_obs_init, m.mu, m.sigma, m.mu_hat, m.sigma_hat, m.zz], feed_dict=feed_dict)

    print('Reconstruction loss: ', rec_loss_)
    print('Regularization loss: ', reg_loss_)
    #print('Total loss: ', loss_)


In [ ]:
_ = plt.hist(zz_.flatten(),bins=np.arange(0,10,0.1))

In [ ]:
def kl_divergence_gaussians(q_mu, q_sigma, p_mu, p_sigma):
    r = q_mu - p_mu
    a = np.log(p_sigma) - np.log(q_sigma)
    return  - .5 * (1. - (q_sigma**2 + r**2) / p_sigma**2)

In [ ]:
def kl_divergence_gaussians(q_mu, q_sigma, p_mu, p_sigma):
    r = q_mu - p_mu
    a = np.log(p_sigma) - np.log(q_sigma)
    b = q_sigma**2/p_sigma**2
    c = r**2/p_sigma**2
    return a - .5 * (1. - (b + c)), a,b,c

In [ ]:
def kl_divergence_gaussians_log(q_mu, q_sigma_log, p_mu, p_sigma_log):
    r = q_mu - p_mu
    return np.sum(p_sigma_log - q_sigma_log - .5 * (1. - (np.exp(q_sigma_log*2)/np.exp(p_sigma_log*2) + r**2/np.exp(p_sigma_log*2))), axis=-1)

In [ ]:
kl, a,b,c = kl_divergence_gaussians(mu_hat_, sigma_hat_, mu_, sigma_)

In [ ]:
kl_divergence_gaussians_log(mu_hat_, sigma_hat_, mu_, sigma_)

In [ ]:
print(np.mean(np.isnan(a) + np.isinf(a)))
print(np.mean(np.isnan(b) + np.isinf(b)))
print(np.mean(np.isnan(c) + np.isinf(c)))
print(np.mean(np.isnan(kl) + np.isinf(kl)))
